In [2]:
import vitaldb
import pandas as pd
import requests
import json
import io
import time  # Importer time-modulet
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#APTT

#Download clinical information
clinical_information_url = "https://api.vitaldb.net/cases"
df_clinical= pd.read_csv(clinical_information_url)

#File with binary ICU admission infromation
url = 'https://raw.githubusercontent.com/jennyskrytenjohnsen/P10/refs/heads/main/For_machinelearning/number_of_days_in_ICU.csv'
pat_in_ICU = pd.read_csv(url)



#Created a new dataframe for this task with caseid, aptt values and  binary ICu admission info
df_caseidAndVariable =df_clinical[['caseid','preop_aptt']]
df_caseidAndVariableICUInformation = pd.concat([df_caseidAndVariable, pat_in_ICU['icu_days_binary']], axis=1)
df_caseidAndVariableICUInformation=df_caseidAndVariableICUInformation.dropna()

#Allocated space
binary=[]
level_division = []
extreme_level_division = []


#for loop for creating the different data distrubations
for index, row in df_caseidAndVariableICUInformation.iterrows():
    if row['preop_aptt'] < 20:
        extreme_level_division.append('-2')
        level_division.append('-1')
        binary.append('1')
    elif row['preop_aptt'] < 25:
        level_division.append('-1')
        binary.append('1')
        extreme_level_division.append('-1')
    elif 25 <= row['preop_aptt'] <= 35:
        binary.append('0') 
        level_division.append('0')
        extreme_level_division.append('0')
    elif row['preop_aptt'] > 35:
        level_division.append('1')
        binary.append('1')
        extreme_level_division.append('1')
    elif row['preop_aptt'] > 50:
        extreme_level_division.append('2')
        level_division.append('1')
        binary.append('1')

#Add new info to dataframe
df_caseidAndVariableICUInformation['binary'] = binary
df_caseidAndVariableICUInformation['level_division'] = level_division
df_caseidAndVariableICUInformation['extreme_level_division'] = extreme_level_division


print('-------------------------------------------------------------------------------------------')

#Gives a nice overwiev over the new dataframe :)))
print(df_caseidAndVariableICUInformation.head())
###################################################################################################

#Plotes the first plot, you can change the hue variable
sns.countplot(x ='icu_days_binary', hue = "extreme_level_division", data = df_caseidAndVariableICUInformation)
plt.title('aPTT Levels by ICU Admission Status')
plt.ylabel("Patient Counts")
plt.xlabel(" ")
plt.xticks([0, 1], ['No ICU', 'ICU Admitted'])
plt.legend(title='aTTP values (s) ', loc='upper right', labels=['25 < aTTP < 35', 'aTTP > 35', ' aTTP < 25 ', 'aTTP < 20', 'aTTP < 50'])
plt.show()

#Creates a new dataframe for evaluating the recored aptt values
df_atppAndICUBinary= df_caseidAndVariableICUInformation[['preop_aptt', 'icu_days_binary']]

# Split the data
aptt_icu0 = df_atppAndICUBinary[df_atppAndICUBinary['icu_days_binary'] == 0]['preop_aptt']
aptt_icu1 = df_atppAndICUBinary[df_atppAndICUBinary['icu_days_binary'] == 1]['preop_aptt']

print(aptt_icu0)

# Plot both histograms
plt.hist(aptt_icu0, bins=[10, 15, 20, 25, 30, 40, 45, 50, 55],  label='ICU = 0 (Not admitted)', ec='black')
plt.hist(aptt_icu1, bins=[10, 15, 20, 25, 30, 40, 45, 50, 55], label='ICU = 1 (Admitted)', ec='black')

# Labels and legend
plt.xlabel("Preoperative aPTT")
plt.xlim(0,70)
plt.ylabel("Number of Patients")
plt.title("Distribution of APTT by ICU Admission")
plt.legend()
plt.show()



df_caseidAndVariableICUInformation = df_caseidAndVariableICUInformation.drop(columns=['icu_days_binary', 'binary', 'level_division', 'extreme_level_division'])


df_caseidAndVariableICUInformation.rename(columns={
    'caseid': 'CaseID',
    'preop_aptt': 'Variable_PreopAptt'}, inplace=True)

df_caseidAndVariableICUInformation.to_csv('Data_PreopAptt.csv', index=False)




## VASOACTIVE DRUGS

#### PLOTTE KODE ####

# import requests
# import pandas as pd
# import io
# import matplotlib.pyplot as plt

# # List of special variables you're interested in (update as necessary)
# special_variables = ["Orchestra/NEPI_VOL"]

# # Load the tracklist data
# track_list_url = "https://api.vitaldb.net/trks"
# df_tracklist = pd.read_csv(track_list_url)

# # Get the first caseid from the dataframe
# first_caseid = 17

# # Iterate through the tracklist to find the track for the first caseid
# for index, row in df_tracklist.iterrows():
#     if row['tname'] in special_variables and row["caseid"] == first_caseid:
#         trackidentifier = row["tid"]
#         trackdata_url = f"https://api.vitaldb.net/{trackidentifier}"
        
#         # Handle API error
#         try:
#             response = requests.get(trackdata_url, timeout=10)
#             response.raise_for_status()
#         except requests.exceptions.Timeout:
#             print(f"Timeout error: Cannot get {row['tname']} ({trackdata_url})")
#             continue  # Skip to next track

#         try:
#             # Convert API response into a pandas DataFrame
#             trackdata = pd.read_csv(io.StringIO(response.text))
#             if trackdata.empty:
#                 print(f"Track data for track {row['tname']} is empty.")
#                 continue
#         except Exception as e:
#             print(f"Error reading track data for track {row['tname']}: {e}")
#             continue  # Skip if there is an issue reading the track data

#         # Debugging: Check the first few rows of the data
#         print(f"First few rows of track data for caseid {first_caseid}:")
#         print(trackdata.head())

#         # Check if 'Time' and 'Orchestra/NEPI_VOL' columns exist
#         if 'Time' not in trackdata.columns or 'Orchestra/NEPI_VOL' not in trackdata.columns:
#             print(f"Columns 'Time' and/or 'Orchestra/NEPI_VOL' not found in track data.")
#             continue

#         # Extract 'Time' and 'Orchestra/NEPI_VOL' columns
#         time_data = trackdata["Time"]
#         track_values = trackdata["Orchestra/NEPI_VOL"]

#         # Check if the columns have valid data
#         if time_data.empty or track_values.empty:
#             print(f"Empty data for 'Time' or 'Orchestra/NEPI_VOL' in track data for caseid {first_caseid}.")
#             continue

#         # Plot the data
#         plt.figure(figsize=(10, 6))
#         plt.plot(time_data, track_values, label='Orchestra/NEPI_VOL', color='b')
#         plt.xlabel("Time")
#         plt.ylabel("Orchestra/NEPI_VOL Value")
#         plt.title(f"Track Values Over Time for Case ID: {first_caseid}")
#         plt.xticks(rotation=45)
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()

#         break  # We only need the first caseid, so break the loop after plotting
